Fazer um (ou mais) Benchmarks

Preciso ajustar galerkin_method para o primeiro problema variacional da Lista.

Sugestão: criar uma outra função para o funcional 'a' e o 'f' e utilizar dentro de galerkin_method para mais flexibilidade.

In [5]:
import numpy as np
from scipy import integrate
from scipy.misc import derivative

In [6]:
def trapezoidal_rule(func, x_inicial, x_final, num_points):
    x = np.linspace(x_inicial, x_final, num_points)
    y = func(x)
    h = (x_final - x_inicial) / (num_points - 1)
    integral = (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])
    return integral

In [7]:
def gaussian_quad(func, x_inicial, x_final, args=None):
    def_integral = integrate.quad(func, x_inicial, x_final)[0]
    return def_integral

In [8]:
def galerkin_method(basis_functions, fonte, x_inicial, x_final, integration_method, args=None):
    n = len(basis_functions)
    A = np.zeros((n, n))
    l = np.zeros(n)

    for i in range(n):
        for j in range(n):
            A[i, j] = funcional_incognita(basis_functions[i], basis_functions[j], integration_method, x_inicial, x_final, args)
            
        l[i] = funcional_fonte(basis_functions[i], fonte, integration_method, x_inicial, x_final, args)

    coef = np.linalg.solve(A, l)
    return coef

In [9]:
def funcional_incognita(func1, func2, integration_method, x_inicial, x_final, args):
    integrand = lambda x: derivative(func1, x, dx=1e-4) * derivative(func2, x, dx=1e-4)
    return integration_method(integrand, x_inicial, x_final, args)

In [10]:
def funcional_fonte(func, fonte, integration_method, x_inicial, x_final, args):
    integrand = lambda x: func(x) * fonte(x)
    return integration_method(integrand, x_inicial, x_final, args)

In [11]:
basis_func_1 = lambda x: x

basis_func_2 = lambda x: x**2

f = lambda x: np.sin(x)

In [12]:
basis_functions = [basis_func_1, basis_func_2]
x_inicial = 0
x_final = np.pi
num_partitions = 10

# Using Gaussian quadrature
coef_gaussian = galerkin_method(basis_functions, f, x_inicial, x_final, gaussian_quad)
print("Coefficients with Gaussian quadrature:", coef_gaussian)

# Using Trapezoidal rule
coef_trapezoidal = galerkin_method(basis_functions, f, x_inicial, x_final, trapezoidal_rule, num_partitions)
print("Coefficients with Trapezoidal rule:", coef_trapezoidal)

Coefficients with Gaussian quadrature: [ 2.2158542  -0.38701841]
Coefficients with Trapezoidal rule: [ 2.1763078  -0.37766908]


C:\Users\Usuario\AppData\Local\Temp\ipykernel_5604\751507433.py:2: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  integrand = lambda x: derivative(func1, x, dx=1e-4) * derivative(func2, x, dx=1e-4)
